In [1]:
from dotenv import find_dotenv, load_dotenv; load_dotenv(find_dotenv())

True

In [24]:
# %pip install -U langchain langchain_openai
# %pip install -U duckduckgo-search

In [59]:
from typing import Union

In [61]:
from langchain import hub
from langchain_core.messages import AIMessage, HumanMessage, ChatMessage, SystemMessage, FunctionMessage, ToolMessage
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain.agents import AgentExecutor
from langchain.agents.openai_tools.base import create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

In [15]:
model = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.)

In [26]:
# tool_list = [TavilySearchResults(max_results=3)]
tool_list = [DuckDuckGoSearchRun()]

In [62]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
display(prompt)

prompt = ChatPromptTemplate.from_messages([
    ('system','あなたはインターネットに接続されたAIアシスタントです。検索結果に基づき、正確な応答を行なってください。'),
    ('user', '{input}'),
    ('system', '{agent_scratchpad}')
])
prompt.input_types = {
    'chat_history': list[Union[AIMessage, HumanMessage, ChatMessage, SystemMessage, FunctionMessage, ToolMessage]],
    'agent_scratchpad': list[Union[AIMessage, HumanMessage, ChatMessage, SystemMessage, FunctionMessage, ToolMessage]]
}
display(prompt)

/Users/harukary/Documents/workspace/llm_samples/.venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:160: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': list[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': list[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='あなたはインターネットに接続されたAIアシスタントです。検索結果に基づき、正確な応答を行なってください。')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_s

In [46]:
agent = create_openai_tools_agent(model, tool_list, prompt)

In [47]:
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)

In [48]:
res = agent_executor.invoke({"input": "じゃがいもについて栄養成分や旬を教えて"})



> Entering new AgentExecutor chain...


KeyError: "Input to ChatPromptTemplate is missing variables {'user_input', 'chat_history'}.  Expected: ['agent_scratchpad', 'chat_history', 'user_input'] Received: ['input', 'intermediate_steps', 'agent_scratchpad']"

In [31]:
print(res['output'])

じゃがいもの栄養成分は、ビタミンCやカリウムが豊富に含まれています。100g中には、ビタミンCが35mg、カリウムが410mg含まれています。また、じゃがいもは多くのでんぷんを含んでいますが、カロリーはお米の半分以下です。

じゃがいもの旬は6月から9月です。この時期には新じゃがいもが出回ります。

以上がじゃがいもの栄養成分と旬についての情報です。
